In [4]:
import tensorflow

from tensorflow.keras.applications import ResNet50

# 创建模型
model = ResNet50(weights='imagenet', include_top=False, input_shape=(512, 512, 3))


for layer in model.layers:
    if "out" in layer.name or 'conv1_relu' in layer.name:
        print('Layer Name:', layer.name)
        print('Feature Map Shape:', layer.output_shape)


Layer Name: conv1_relu
Feature Map Shape: (None, 256, 256, 64)
Layer Name: conv2_block1_out
Feature Map Shape: (None, 128, 128, 256)
Layer Name: conv2_block2_out
Feature Map Shape: (None, 128, 128, 256)
Layer Name: conv2_block3_out
Feature Map Shape: (None, 128, 128, 256)
Layer Name: conv3_block1_out
Feature Map Shape: (None, 64, 64, 512)
Layer Name: conv3_block2_out
Feature Map Shape: (None, 64, 64, 512)
Layer Name: conv3_block3_out
Feature Map Shape: (None, 64, 64, 512)
Layer Name: conv3_block4_out
Feature Map Shape: (None, 64, 64, 512)
Layer Name: conv4_block1_out
Feature Map Shape: (None, 32, 32, 1024)
Layer Name: conv4_block2_out
Feature Map Shape: (None, 32, 32, 1024)
Layer Name: conv4_block3_out
Feature Map Shape: (None, 32, 32, 1024)
Layer Name: conv4_block4_out
Feature Map Shape: (None, 32, 32, 1024)
Layer Name: conv4_block5_out
Feature Map Shape: (None, 32, 32, 1024)
Layer Name: conv4_block6_out
Feature Map Shape: (None, 32, 32, 1024)
Layer Name: conv5_block1_out
Feature Map 

In [7]:
import os
import cv2
import glob
import random
import shutil

# split train val test
root_dir = "../data/"
filenames = glob.glob(root_dir + '*T.png')
random.shuffle(filenames)

train_dir = os.path.join(root_dir, "train")
val_dir = os.path.join(root_dir, "val")
test_dir = os.path.join(root_dir, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

train_img = os.path.join(train_dir, "images")
train_mask = os.path.join(train_dir, "mask_ori")
val_img = os.path.join(val_dir, "images")
val_mask = os.path.join(val_dir, "mask_ori")
test_img = os.path.join(test_dir, "images")
test_mask =os.path.join(test_dir, "mask_ori")
os.makedirs(train_img, exist_ok=True)
os.makedirs(train_mask, exist_ok=True)
os.makedirs(val_img, exist_ok=True)
os.makedirs(val_mask, exist_ok=True)
os.makedirs(test_img, exist_ok=True)
os.makedirs(test_mask, exist_ok=True)

length = len(filenames)
train_size = int(length*0.8)
val_size = int(length*0.9)

for i in range(train_size):
    shutil.copy(filenames[i], os.path.join(train_img, os.path.basename(filenames[i])))
    shutil.copy(filenames[i][:-5]+"A.png", os.path.join(train_mask, os.path.basename(filenames[i][:-5]+"A.png")))
for i in range(train_size,val_size):
    shutil.copy(filenames[i], os.path.join(val_img, os.path.basename(filenames[i])))
    shutil.copy(filenames[i][:-5]+"A.png", os.path.join(val_mask, os.path.basename(filenames[i][:-5]+"A.png")))
for i in range(val_size,length):
    shutil.copy(filenames[i], os.path.join(test_img, os.path.basename(filenames[i])))
    shutil.copy(filenames[i][:-5]+"A.png", os.path.join(test_mask, os.path.basename(filenames[i][:-5]+"A.png")))

files = glob.glob(train_img+"/*.png")
print(f"trainset size = {len(files)}")
files = glob.glob(val_img +"/*.png")
print(f"valset size = {len(files)}")
files = glob.glob(test_img+"/*.png")
print(f"testset size = {len(files)}")



trainset size = 224
valset size = 28
testset size = 28


In [13]:
import cv2
import numpy as np

img = cv2.imread('../data/test/mask_ori/262A.png')
print(img.shape)
print(np.unique(np.unique(img, axis=0), axis=0))
print(img[0][0])

(512, 512, 3)
[[[  0 255   0]
  [  0 255   0]
  [  0 255   0]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]

 [[  0 255   0]
  [  0 255   0]
  [  0 255   0]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]

 [[  0 255   0]
  [  0 255   0]
  [  0 255   0]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]

 ...

 [[ 75  25 230]
  [ 75  25 230]
  [ 75  25 230]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]

 [[ 75  25 230]
  [ 75  25 230]
  [ 75  25 230]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]

 [[ 75  25 230]
  [ 75  25 230]
  [ 75  25 230]
  ...
  [  0 255   0]
  [  0 255   0]
  [  0 255   0]]]
[  0 255   0]


In [8]:
import os
import cv2
import glob
import numpy as np

def map_rgb_to_id(mask):
    mask_label = np.zeros(mask.shape[:2], dtype=np.int32)
    # Define the mapping from class to RGB color
    label_to_color = {
        0: (75, 25, 230),  # Red, class 1
        1: (0, 255, 0),    # Green, class 2
        2: (255, 255, 0),   # Blue, class 3
        3: (0, 0, 0)
    }

    # Map each class in the class_map to its corresponding color
    for class_id, color in label_to_color.items():
        mask_label[(mask == color).all(axis=-1)] = class_id

    return mask_label
# visualize
root_dir ="../data/original_dataset/"
target_dir = "../data/visualization"
os.makedirs(target_dir, exist_ok=True)
filenames = glob.glob(root_dir + '*T.png')
for filename in filenames:
    maskname = filename[:-5] + "A.png"
    img = cv2.imread(filename)
    mask = cv2.imread(maskname)
    mask = map_rgb_to_id(mask)
    
    img[..., 0] = np.where(mask == 0, 255, img[..., 0] )
    img[..., 1] = np.where(mask == 1, 255, img[..., 1] )
    img[..., 2] = np.where(mask == 2, 255, img[..., 2] )
    img[..., 0] = np.where(mask == 3, 0, img[..., 0] )
    img[..., 1] = np.where(mask == 3, 0, img[..., 1] )
    img[..., 2] = np.where(mask == 3, 0, img[..., 2] )
    
    cv2.imwrite(os.path.join(target_dir, os.path.basename(filename)), img)
    
    
    

(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)